In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import img_to_array, load_img
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import time
%load_ext tensorboard

datagen = ImageDataGenerator(
        rescale=1/255.0,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='reflect')

generator = datagen.flow_from_directory(
    r"flowers",
    target_size=(150,150),
    batch_size=16,
    class_mode='sparse'
)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Found 4317 images belonging to 5 classes.


In [7]:
dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]
SIZE = 150
INPUT_SHAPE = (SIZE, SIZE, 3) 

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
      tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
      
      print(NAME)
      #model initialzation
      model = Sequential()

      #model definition
      model.add(Conv2D(layer_size, (3,3), input_shape = INPUT_SHAPE))
      model.add(Activation('relu'))  
      model.add(MaxPooling2D(pool_size=(2,2)))

      for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3,3)))
        model.add(Activation('relu')) 
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      model.add(Flatten())

      for l in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu')) 


      #final output layer
      model.add(Dense(5))
      model.add(Activation('softmax'))

      #settings
      model.compile(loss='sparse_categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
      #fit
      model.fit(generator, batch_size=64, epochs=25, callbacks=[tensorboard])

3-conv-64-nodes-1-dense-1668763596
Epoch 1/25
270/270 [==============================] - 56s 203ms/step - loss: 1.2894 - accuracy: 0.4320
Epoch 2/25
270/270 [==============================] - 26s 96ms/step - loss: 1.1132 - accuracy: 0.5462
Epoch 3/25
270/270 [==============================] - 25s 93ms/step - loss: 1.0050 - accuracy: 0.5944
Epoch 4/25
270/270 [==============================] - 25s 93ms/step - loss: 0.9406 - accuracy: 0.6354
Epoch 5/25
270/270 [==============================] - 27s 99ms/step - loss: 0.8877 - accuracy: 0.6627
Epoch 6/25
270/270 [==============================] - 26s 97ms/step - loss: 0.8586 - accuracy: 0.6646
Epoch 7/25
270/270 [==============================] - 30s 112ms/step - loss: 0.8131 - accuracy: 0.6938
Epoch 8/25
270/270 [==============================] - 31s 115ms/step - loss: 0.7930 - accuracy: 0.6975
Epoch 9/25
270/270 [==============================] - 26s 97ms/step - loss: 0.7718 - accuracy: 0.7042
Epoch 10/25
270/270 [=======================

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
activation_5 (Activation)    (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
activation_6 (Activation)    (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)       

In [9]:
model.save('Flowers.h5')

In [11]:
import cv2

CATEGORIES = ['Daisy', 'Dandelion', 'Rose', 'Sunflower', 'Tulip']

def prepare(filepath):
  IMG_SIZE = 150
  img_array = cv2.imread(filepath)
  new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
  return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [13]:
prediction = model.predict([prepare(r"flowers/dandelion/7355522_b66e5d3078_m.jpg")])
print(CATEGORIES[int(prediction[0][0])])

Daisy
